In [14]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os

# --- Configuration ---
# Apply a clean seaborn style suitable for papers/reports
sns.set_theme(style="whitegrid", font_scale=1.1, rc={"grid.alpha": 0.3})
plt.rcParams['font.family'] = 'sans-serif' # Use a clean sans-serif font
# plt.rcParams['font.sans-serif'] = ['Arial', 'Helvetica', 'DejaVu Sans'] # Specify preferred fonts
# plt.rcParams['text.usetex'] = False # Set to True if LaTeX is installed and desired for math text

# Define a professional and colorblind-friendly color palette
colors = {
    "DMFD - Image": sns.color_palette("colorblind")[0], # Blue
    "DMFD - State": sns.color_palette("colorblind")[1], # Orange
    "BC - Image": sns.color_palette("colorblind")[2], # Green
    "BC - State": sns.color_palette("colorblind")[3], # Red
    "BC-Diffusion - Image": sns.color_palette("colorblind")[4], # Purple
    "BC-Diffusion - State": sns.color_palette("colorblind")[5], # Brown
}

# Define consistent plot parameters
plot_params = {
    'linewidth': 2,
    'markersize': 7,
    'marker': 'o',
    'linestyle': '-',
}
fill_alpha = 0.15
label_fontsize = 12
title_fontsize = 14
legend_fontsize = 11
tick_fontsize = 10
annotation_fontsize = 9
annotation_offset_y = 8 # Base offset for annotations

output_dir = 'img/cloth_fold'
os.makedirs(output_dir, exist_ok=True) # Ensure output directory exists

# --- Data ---
# TASK: Cloth Fold.
# y-axis: End-of-training normalized performance.
# x-axis: Number of demonstrations (log scale), constant 8:1 demo:variation ratio.
number_of_demonstrations = np.array([40, 200, 1000, 4000, 8000])

# Store data in a dictionary for easier management and plotting
# Using np.nan for missing/invalid data points
data = {
    # DMFD - IMAGE
    "DMFD - Image": (np.array([0,    0.021, 0,     0.33,  0.63]),
                     np.array([0,    0,     0,     0.304, 0.272])),
    # DMFD - STATE
    "DMFD - State": (np.array([-0.10, 0.1815, 0.5155, 0.5128, 0.5275]),
                     np.array([ 0.605, 0.400,  0.288,  0.288,  0.244])),
    # BC - IMAGE
    "BC - Image":   (np.array([-0.0883, -0.0537, 0.0859, 0.1470, 0.1989]),
                     np.array([ 0.1708,   0.3245, 0.2653, 0.4167, 0.3165])),
    # BC - STATE
    "BC - State":   (np.array([ 0.0000, -0.1264, -0.1622, 0.3067, 0.6402]),
                     np.array([ 0.0000,  0.4087,  0.8797, 0.6213, 0.2949])),
    # BC-Diffusion - STATE
    "BC-Diffusion - State": (np.array([0.4340, 0.7398, 0.7298, 0.6996, 0.7005]),
                             np.array([0.3212, 0.1015, 0.1129, 0.1395, 0.1890])),
    # BC-Diffusion - IMAGE
    "BC-Diffusion - Image": (np.array([-0.3294, -0.0718, 0.5268, 0.6602, 0.7010]),
                             np.array([0.4616, 0.3448, 0.3583, 0.2757, 0.2090])),
}

# --- Plotting Function ---
def plot_performance(data_keys, title, filename):
    """
    Generates and saves a performance plot for the specified data keys.

    Args:
        data_keys (list): List of keys from the `data` dictionary to plot.
        title (str): The title for the plot.
        filename (str): The path to save the plot image.
    """
    plt.figure(figsize=(7, 4.5)) # Consistent figure size
    ax = plt.gca()

    for i, key in enumerate(data_keys):
        if key not in data:
            print(f"Warning: Data key '{key}' not found. Skipping.")
            continue

        perf, std = data[key]
        color = colors.get(key, 'black') # Use defined color or default to black

        # Filter out NaN values for plotting lines and shaded regions
        valid_mask = ~np.isnan(perf)
        x_plot = number_of_demonstrations[valid_mask]
        y_plot = perf[valid_mask]
        s_plot = std[valid_mask]

        if len(x_plot) > 0: # Only plot if there's valid data
            # Plot the main performance line
            ax.plot(x_plot, y_plot,
                    marker=plot_params['marker'],
                    linestyle=plot_params['linestyle'],
                    linewidth=plot_params['linewidth'],
                    markersize=plot_params['markersize'],
                    color=color,
                    label=key)

            # Plot the shaded standard deviation region
            ax.fill_between(x_plot, y_plot - s_plot, y_plot + s_plot,
                            alpha=fill_alpha, color=color, linewidth=0) # No border for fill

            # Add annotations for each data point
            for x_val, y_val in zip(x_plot, y_plot):
                 # Adjust vertical offset slightly based on line index to reduce overlap
                offset = annotation_offset_y * (1 if i % 2 == 0 else -1.5)
                ax.annotate(f"{y_val:.2f}", (x_val, y_val),
                            textcoords="offset points",
                            xytext=(0, offset),
                            ha='center',
                            fontsize=annotation_fontsize,
                            color=color) # Match annotation color to line for clarity

    # --- Plot Customization ---
    ax.set_xscale('log')
    ax.set_xlabel('Number of Demonstrations (Log Scale)', fontsize=label_fontsize)
    ax.set_ylabel('Normalized Performance', fontsize=label_fontsize)
    ax.set_title(title, fontsize=title_fontsize, weight='bold')

    # Set x-axis ticks to match the data points
    ax.set_xticks(number_of_demonstrations)
    ax.set_xticklabels([str(x) for x in number_of_demonstrations], fontsize=tick_fontsize)
    ax.tick_params(axis='y', labelsize=tick_fontsize)

    # Set y-axis limits for consistency across plots
    ax.set_ylim(-0.2, 1.05)

    # Add legend
    ax.legend(fontsize=legend_fontsize, frameon=True, loc='best') # 'best' location

    # Final adjustments and save
    plt.tight_layout(pad=1.0) # Add padding to prevent clipping
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close() # Close the figure to free memory
    print(f"Plot saved to {filename}")


# --- Generate Plots ---
# 1) DMFD Performance Plot
plot_performance(
    data_keys=["DMFD - Image", "DMFD - State"],
    title="DMFD Performance vs. Demonstrations",
    filename=os.path.join(output_dir, 'dmfd_performance_vs_num_dems.png')
)

# 2) BC-Diffusion Performance Plot
plot_performance(
    data_keys=["BC-Diffusion - Image", "BC-Diffusion - State"],
    title="BC-Diffusion Performance vs. Demonstrations",
    filename=os.path.join(output_dir, 'bc_diffusion_performance_vs_num_dems.png')
)

# 3) BC Performance Plot
plot_performance(
    data_keys=["BC - Image", "BC - State"],
    title="BC Performance vs. Demonstrations",
    filename=os.path.join(output_dir, 'bc_performance_vs_num_dems.png')
)


# --- Print BC-Diffusion Table ---
print("\n--- BC-Diffusion Performance (State) ---")
# Use headers that match the plot labels for consistency
print(f"{'Num Demos':<12} {'Performance':<15} {'Std Dev':<10}")
print("-" * (12 + 1 + 15 + 1 + 10)) # Dynamic separator line length
bc_diff_state_perf, bc_diff_state_std = data["BC-Diffusion - State"]
for n, perf, std in zip(number_of_demonstrations, bc_diff_state_perf, bc_diff_state_std):
    # Handle potential NaN values in printing as well
    perf_str = f"{perf:<15.4f}" if not np.isnan(perf) else f"{'N/A':<15}"
    std_str = f"{std:<10.4f}" if not np.isnan(std) else f"{'N/A':<10}"
    print(f"{n:<12} {perf_str} {std_str}")
print("-" * (12 + 1 + 15 + 1 + 10))



Plot saved to img/cloth_fold/dmfd_performance_vs_num_dems.png
Plot saved to img/cloth_fold/bc_diffusion_performance_vs_num_dems.png
Plot saved to img/cloth_fold/bc_performance_vs_num_dems.png

--- BC-Diffusion Performance (State) ---
Num Demos    Performance     Std Dev   
---------------------------------------
40           0.4340          0.3212    
200          0.7398          0.1015    
1000         0.7298          0.1129    
4000         0.6996          0.1395    
8000         0.7005          0.1890    
---------------------------------------


In [6]:
import numpy as np

# MANUAL DATA GATHERING
diff_state_performances = [-0.3294, -0.0718, 0.5268, 0.6602, 0.7010]
diff_state_stds = [0.4616, 0.3448, 0.3583, 0.2757, 0.2090]

all_perf = np.array([
0.0000, 0.0000, 0.4823, 0.0000, 0.0000, 0.0000, 0.2673, 0.0000, 0.6917, 0.3802,  0.7142,  0.0000,  0.7028,  0.0000,  0.0000,  0.5328,  0.6315,  0.0000,  0.0000,  0.7042, 0.0000, 0.7090, 0.0000, -0.2520, 0.7411, 0.0000, 0.6510, -0.3880, 0.0000, 0.7388,  0.0000,  0.7253,  0.0000,  0.4145,  0.6715,  0.0000,  0.6485,  0.2255,  0.3094,  0.6985
])
print('\n!!!!!!! Final Normalized Performance Statistics (100 episodes over 5 seeds) !!!!!!!')
print(f'Mean: {np.mean(all_perf):.4f}')
print(f'Std: {np.std(all_perf):.4f}')
print(f'Median: {np.median(all_perf):.4f}')
print(f'25th Percentile: {np.percentile(all_perf, 25):.4f}')
print(f'75th Percentile: {np.percentile(all_perf, 75):.4f}')
print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')


!!!!!!! Final Normalized Performance Statistics (100 episodes over 5 seeds) !!!!!!!
Mean: 0.2750
Std: 0.3363
Median: 0.1128
25th Percentile: 0.0000
75th Percentile: 0.6561
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


Variations differ in: 

- Cloth Size: If vary_cloth_size=True (default), each environment will have a randomly sampled cloth size using _sample_cloth_size() which returns different dimensions.
- Rotation: Each cloth is rotated by a random angle between -π/4 and π/4 radians (unless num_variations=1, in which case no rotation is applied).
- Position: The cloth is centered and stabilized in each variation, but the initial settling of the cloth can create subtle differences in the starting state.
